<a href="https://colab.research.google.com/github/PsorTheDoctor/Sekcja-SI/blob/master/neural_networks/RNN/seq2seq_math.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Seq2Seq: Dodawanie liczb

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

TRAINING_SIZE = 50000
DIGITS = 3  # liczba cyfr składnika
REVERSE = True

# Maksymalna długość wejścia np. '345+678'
MAXLEN = DIGITS + 1 + DIGITS

##Generowanie danych

In [8]:
class CharacterTable:
  
  def __init__(self, chars):
    self.chars = sorted(set(chars))
    self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
    self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

  def encode(self, C, num_rows):
    x = np.zeros((num_rows, len(self.chars)))
    for i, c in enumerate(C):
      x[i, self.char_indices[c]] = 1
    return x

  def decode(self, x, calc_argmax=True):
    if calc_argmax:
      x = x.argmax(axis=-1)
    return ''.join(self.indices_char[x] for x in x)

# Wszystkie liczby, znaki i spacja
chars = '0123456789+-* '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generowanie danych...')
while len(questions) < TRAINING_SIZE:
  f = lambda: int(
      ''.join(
          np.random.choice(list('0123456789'))
          for i in range(np.random.randint(1, DIGITS + 1))
      )
  )
  a, b = f(), f()
  sign = np.random.choice(list('-+*'))
  # Pominięcie przykładu, który był już widziany
  # Także pominięcie wszystkiego w rodzaju x+Y == Y+x
  key = tuple(sorted((a, b)))
  if key in seen:
    continue
  seen.add(key)
  
  q = '{}{}{}'.format(a, sign, b)
  # Padding danych przez spacje
  query = q + ' ' * (MAXLEN - len(q))
  if sign == '+':
    ans = str(a + b)
  elif sign == '-':
    ans = str(a - b)
  elif sign == '*':
    ans = str(a * b)
  # Odpowiedzi mogą mieć maksymalną długość DIGITS + 3
  ans += ' ' * (DIGITS + 3 - len(ans))
  if REVERSE:
    # Odwrócenie zapytania, np. '12+345  ' staje się '  543+21'
    # Spacje są użyte jako padding
    query = query[::-1]
  questions.append(query)
  expected.append(ans)
print('Liczba przykładów:', len(questions))

Generowanie danych...
Liczba przykładów: 50000


##Wektoryzacja danych

In [11]:
print('Wektoryzacja...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 3, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
  x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
  y[i] = ctable.encode(sentence, DIGITS + 3)

indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Dane treningowe:')
print(x_train.shape)
print(y_train.shape)

print('Dane walidacyjne:')
print(x_val.shape)
print(y_val.shape)

Wektoryzacja...
Dane treningowe:
(45000, 7, 14)
(45000, 6, 14)
Dane walidacyjne:
(5000, 7, 14)
(5000, 6, 14)


##Budowa modelu

In [19]:
print('Budowa modelu...')
num_layers = 3

model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 3))

for _ in range(num_layers):
  model.add(layers.LSTM(128, return_sequences=True))

model.add(layers.Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Budowa modelu...
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 128)               73216     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
lstm_9 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
lstm_10 (LSTM)               (None, 6, 128)            131584    
_________________________________________________________________
dense_3 (Dense)              (None, 6, 14)             1806      
Total params: 469,774
Trainable params: 469,774
Non-trainable params: 0
_______________________________

##Trening modelu

In [21]:
epochs = 50
batch_size = 32

for epoch in range(1, epochs + 1):
  print()
  print('Iteracja', epoch)
  model.fit(
      x_train,
      y_train,
      batch_size=batch_size,
      epochs=1,
      validation_data=(x_val, y_val)
  )
  # Wybór 10 losowych próbek ze zbioru walidacyjnego, 
  # abyśmy mogli zobaczyć błędy
  for i in range(10):
    ind = np.random.randint(0, len(x_val))
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = np.argmax(model.predict(rowx), axis=-1)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
      print('☑ ' + guess)
    else:
      print('☒ ' + guess)


Iteracja 1
1407/1407 [==============================] - 13s 10ms/step - loss: 0.4927 - accuracy: 0.7964 - val_loss: 0.5015 - val_accuracy: 0.7902
Q 678-86  T 592    ☒ 598   
Q 93-12   T 81     ☒ 79    
Q 656+318 T 974    ☒ 976   
Q 569+591 T 1160   ☑ 1160  
Q 635-32  T 603    ☒ 605   
Q 71+775  T 846    ☒ 840   
Q 17*637  T 10829  ☒ 11909 
Q 202-871 T -669   ☒ -667  
Q 469-181 T 288    ☒ 280   
Q 212-149 T 63     ☒ 65    

Iteracja 2
1407/1407 [==============================] - 13s 9ms/step - loss: 0.4818 - accuracy: 0.8019 - val_loss: 0.5274 - val_accuracy: 0.7799
Q 528-569 T -41    ☒ 53    
Q 0-248   T -248   ☒ -352  
Q 258-343 T -85    ☒ -81   
Q 725*92  T 66700  ☒ 67720 
Q 422*794 T 335068 ☒ 332728
Q 5*348   T 1740   ☒ 1800  
Q 96-822  T -726   ☑ -726  
Q 68-390  T -322   ☒ -324  
Q 341-272 T 69     ☒ 79    
Q 18-43   T -25    ☒ -29   

Iteracja 3
1407/1407 [==============================] - 13s 9ms/step - loss: 0.4763 - accuracy: 0.8063 - val_loss: 0.5077 - val_accuracy: 0.7890
Q